# Who's winning it? Forecasting sports tournaments

*S3, Pozega, Croatia. July 19-27, 2017*

The goal of this project is to build a probabilistic forecast of the EURO 2016 football tournament using the historical data about international football matches. The data are used to estimate team strength and to build a goal prediction model. The forecast probabilities are estimated with Monte Carlo simulation.

## Building the data-driven components of the system

We have got the scores for all international football matches since 1870s. We first use these data to estimate team strength using the Elo ranking method. Then, we use the Elo ratings to build a goal prediction model.

### Loading the data

In [33]:
import pandas as pd

In [34]:
scores = pd.read_csv('international+euro16.csv',
                     names=['type_long', 'type', 'date', 'team1', 'team2', 'goals1', 'goals2'],
                     sep=',', header=None,
                     parse_dates=[2], index_col=[2])

In [35]:
print(scores.shape)
print(scores.dtypes)
scores.head()

(39279, 6)
type_long    object
type         object
team1        object
team2        object
goals1        int64
goals2        int64
dtype: object


,type_long,type,team1,team2,goals1,goals2
date,,,,,,
1872-11-30,Friendly,F,Scotland,England,0,0
1873-08-03,Friendly,F,England,Scotland,4,2
1874-07-03,Friendly,F,Scotland,England,2,1
1875-06-03,Friendly,F,England,Scotland,2,2
1876-04-03,Friendly,F,Scotland,England,3,0


### Building the Elo ranking

The Elo rating of a team represents its strength. The probability that Team 1 with rating $R_1$ beats Team 2 with rating $R_2$ at home is calculated as follows: $$P(\text{Team 1 beats Team 2}) = \frac{1}{1+{10}^{-(R_1+H-R_2)/400}}$$

The rating of a team is updated after each match based on the strength of the opponent, the outcome, and a number of parameters: $$\Delta R_1=(O_\text{actual}-O_\text{expected}) \times K \times I \times N \times G$$

In [36]:
Rinitial=1500

def expected_outcome(r, ro):
    return  1/(1+10**((ro-r)/400))

def probabilities(exp_outcome):
    p_draw = exp_outcome * (1 - exp_outcome) * 4/3
    p_win = exp_outcome - 0.5 * p_draw
    p_loss = 1 - p_draw - p_win
    return { 1: p_win, 0.5: p_draw, 0: p_loss }

def outcome(g,go):
    if g > go:
        outcome = 1
    elif g < go:
        outcome = 0
    else:
        outcome = 0.5
    
    return (outcome)

def updateElo(r,ro,g,go,game_type,K,I):
    actual_outcome = outcome(g,go)
    expected_outcome_ = expected_outcome(r,ro)
    
    if g >= 5:
        N = 1
    elif g == 4:
        N = 0.85
    elif g == 3:
        N = 0.60
    elif g == 2:
        N = 0.50
    else:
        N = 0.40
    
    goal_difference = abs(g-go)
    if goal_difference >= 5:
        G=1.5
    elif goal_difference == 4:
        G=1.35
    elif goal_difference == 3:
        G=1.20
    elif goal_difference == 2:
        G=1.05
    else:
        G=1
    
    return (actual_outcome - expected_outcome_)*K*I[game_type]*N*G

def newElo(r1,r2,g1,g2,game_type,K,I,H):
    if game_type == 'Friendly' or 'Qualifier' in game_type:
        r1_ = r1 + H
    else:
        r1_ = r1
        
    return r1 + updateElo(r1_,r2,g1,g2,game_type,K,I), r2 + updateElo(r2,r1_,g2,g1,game_type,K,I)

def eloRatings(scores,K,I,H,return_elo_columns=False):
    '''Building Elo ratings using `scores` as the training data and `K` and `H` as parameters'''
    elos = {}
    elos1 = []
    elos2 = []

    for index, row in scores.iterrows():
        t1 = row['team1']
        t2 = row['team2']
        r1 = elos.get(t1, Rinitial)
        r2 = elos.get(t2, Rinitial)
        
        if return_elo_columns:
            elos1.append(r1)
            elos2.append(r2)
        
        r1_new, r2_new = newElo(r1, r2, row['goals1'], row['goals2'],row['type_long'],K,I,H)
        elos[t1] = r1_new
        elos[t2] = r2_new

    return elos, elos1, elos2

def top_elo_teams(ratings,k=10):
    '''Return top-`k` teams according to Elo ratings in `ratings`'''
    by_rating = sorted(ratings.keys(), key=lambda country: ratings[country], reverse=True)
    return [(country, ratings[country]) for country in by_rating[0:k]]

In [37]:
equal_game_type_weights = {
    'Africa':1 ,
    'AfricaQualifier':1 ,
    'America':1 ,
    'AmericaQualifier':1 ,
    'Asia':1 ,
    'AsiaQualifier':1 ,
    'Confederations':1 ,
    'ConfederationsQualifier':1 ,
    'Europe':1 ,
    'EuropeQualifier':1 ,
    'Friendly':1 ,
    'Oceania':1 ,
    'OceaniaQualifier':1 ,
    'SouthAmerica':1 ,
    'SouthAmericaQualifier':1 ,
    'World':1 , 
    'WorldQualifier':1
}

In [38]:
elo1, elo2 = [1767, 1686]
print(expected_outcome(elo1, elo2))
print(newElo(elo1, elo2, 2, 1, 'Friendly', K=30, I=equal_game_type_weights, H=0))

0.6145013610085578
(1772.7824795848717, 1681.3740163321027)


#### Tuning the parameters

We split the entire data into training data and validation data. We built the ratings using the training set and pick the parameter values that yield the lowest prediction error on the validation set as measured by *log loss*.

In [39]:
train_end   = '2013-12-31'

validation_start = '2014-01-01'
validation_end   = '2016-06-08'
validation_set   = scores[validation_start:validation_end]
validation_size  = validation_set.shape[0]

test_start = '2016-06-10'
test_set   = scores[test_start:]
test_size  = test_set.shape[0]

validation_size, test_size

(2260, 51)

In [40]:
from math import log10

def logLoss(p, eps=10**(-15)):
    p_nonzero = max(min(p, 1-eps), eps)
    return log10(p_nonzero)

In [41]:
random_guess_loss = -logLoss(1/3)
random_guess_loss

0.47712125471966244

In [42]:
def elo_log_loss(ratings_, matches, K, I, H):
    num_matches = matches.shape[0]
    
    ratings=ratings_.copy()
    log_loss = 0
    for index, row in matches.iterrows():
        t1 = row['team1']
        t2 = row['team2']
        r1 = ratings.get(t1, Rinitial)
        r2 = ratings.get(t2, Rinitial)
        
        g1 = row['goals1']
        g2 = row['goals2']
        
        actual_outcome=outcome(g1,g2)
        forecast=probabilities(expected_outcome(r1,r2))[actual_outcome]
        log_loss = log_loss + logLoss(forecast)
        
        r1_new, r2_new = newElo(r1, r2, row['goals1'], row['goals2'], row['type_long'], K=K, I=I, H=H)
        ratings[t1] = r1_new
        ratings[t2] = r2_new
        
    return -log_loss / num_matches

In [43]:
different_type_weights = [
    equal_game_type_weights,
    {'Africa': 1.25,
     'AfricaQualifier': 1,
     'America': 1.25,
     'AmericaQualifier': 1,
     'Asia': 1.25,
     'AsiaQualifier': 1,
     'Confederations': 2,
     'ConfederationsQualifier': 2,
     'Europe': 3,
     'EuropeQualifier': 2.5,
     'Friendly': 1,
     'Oceania': 1.5,
     'OceaniaQualifier': 1.25,
     'SouthAmerica': 2.75,
     'SouthAmericaQualifier': 2.4,
     'World': 3.5,
     'WorldQualifier': 2.75,
     },
    {'Africa': 0.6,
     'AfricaQualifier': 0.35,
     'America': 0.6,
     'AmericaQualifier': 0.35,
     'Asia': 0.6,
     'AsiaQualifier': 0.35,
     'Confederations': 1,
     'ConfederationsQualifier': 1,
     'Europe': 1.25,
     'EuropeQualifier': 1,
     'Friendly': 0.25,
     'Oceania': 0.5,
     'OceaniaQualifier': 0.25,
     'SouthAmerica': 1.15,
     'SouthAmericaQualifier': 0.85,
     'World': 1.5,
     'WorldQualifier': 1,
     },
    {'Africa': 3,
     'AfricaQualifier': 2,
     'America': 2.5,
     'AmericaQualifier': 1.75,
     'Asia': 2.5,
     'AsiaQualifier': 1.75,
     'Confederations': 4,
     'ConfederationsQualifier': 4,
     'Europe': 4.5,
     'EuropeQualifier': 3.5,
     'Friendly': 1.5,
     'Oceania': 2.5,
     'OceaniaQualifier': 1.75,
     'SouthAmerica': 4.75,
     'SouthAmericaQualifier': 3.85,
     'World': 5,
     'WorldQualifier': 4,
     },
    {'Africa': 1.25,
     'AfricaQualifier': 1,
     'America': 1.25,
     'AmericaQualifier': 1,
     'Asia': 1.25,
     'AsiaQualifier': 1,
     'Confederations': 2,
     'ConfederationsQualifier': 2,
     'Europe': 2.5,
     'EuropeQualifier': 2,
     'Friendly': 1,
     'Oceania': 1.5,
     'OceaniaQualifier': 1.25,
     'SouthAmerica': 2.5,
     'SouthAmericaQualifier': 2,
     'World': 3,
     'WorldQualifier': 2.5,
     }
]

In [44]:
# from tqdm import tqdm_notebook as tqdm
def tqdm(x, **kwargs): return x

In [45]:
# parameter_tuning = 'Fixed values'
# parameter_tuning = 'Full search'
parameter_tuning = 'Tune individual parameters'

if parameter_tuning == 'Fixed values':
    best_parameters = [50, 1973, 100, equal_game_type_weights]
    print(best_parameters[:3])
    print(best_parameters[3])
    
    best_K, best_year_start, best_H, best_I = best_parameters
    best_train_start = str(best_year_start) + '-1-1'
    
    best_train_set = scores[best_train_start:train_end]
    best_elos      = eloRatings(best_train_set,K=best_K,I=best_I,H=best_H)[0]
    best_elo_loss  = elo_log_loss(best_elos,validation_set,K=best_K,I=best_I,H=best_H)
    print('\nLog loss on validation set: ELO={0:.4f} RANDOM={1:.4f}'.format(best_elo_loss, random_guess_loss))
elif parameter_tuning == 'Full search':
    best_log_loss = 2
    best_parameters = []

    for K in tqdm([1, 5, 10, 16, 25, 50, 56], desc='K', leave=False):
        for year_start in tqdm(range(1973,2007,5), desc='Start year', leave=False):
            for H in tqdm([0,50,60,100], desc='H', leave=False):
                for I in tqdm(different_type_weights, desc='I', leave=False):
                    train_start = str(year_start) + '-1-1'
                    train_set   = scores[train_start:train_end]

                    elos_initial = eloRatings(train_set,K,I,H)[0]
                    elo_loss     = elo_log_loss(elos_initial,validation_set,K,I,H)

                    if elo_loss < best_log_loss:
                        best_log_loss = elo_loss
                        best_parameters = [K, year_start, H, I]

    print(best_log_loss, random_guess_loss)
    for p in best_parameters:
        print(p)

    best_K, best_year_start, best_H, best_I = best_parameters
    best_train_start = str(best_year_start) + '-1-1'
elif parameter_tuning == 'Tune individual parameters':
    best_log_loss = 2
    best_train_start, best_H, best_I = '1973-1-1', 0, equal_game_type_weights
    
    for K in tqdm([1, 5, 10, 16, 25, 50, 56], desc='K', leave=False):
        train_set   = scores[best_train_start:train_end]

        elos_initial = eloRatings(train_set,K,best_I,best_H)[0]
        elo_loss     = elo_log_loss(elos_initial,validation_set,K,best_I,best_H)

        if elo_loss < best_log_loss:
            best_log_loss = elo_loss
            best_K = K
    print('            ERROR  VALUE')
    print('Best K:     {1:.4f} {0:d}'.format(best_K, best_log_loss))
    
    for year_start in tqdm(range(1974,2007,5), desc='Start year', leave=False):
        train_start = str(year_start) + '-1-1'
        train_set   = scores[train_start:train_end]

        elos_initial = eloRatings(train_set,best_K,best_I,best_H)[0]
        elo_loss     = elo_log_loss(elos_initial,validation_set,best_K,best_I,best_H)

        if elo_loss < best_log_loss:
            best_log_loss = elo_loss
            best_train_start = str(year_start) + '-1-1'
    print('Best start: {1:.4f} {0:s}'.format(best_train_start, best_log_loss))
    
    for H in tqdm([0,50,60,100], desc='H', leave=False):
        train_set   = scores[best_train_start:train_end]

        elos_initial = eloRatings(train_set,best_K,best_I,H)[0]
        elo_loss     = elo_log_loss(elos_initial,validation_set,K,best_I,H)

        if elo_loss < best_log_loss:
            best_log_loss = elo_loss
            best_H = H
    print('Best H:     {1:.4f} {0:d}'.format(best_H, best_log_loss))
        
    for I in tqdm(different_type_weights, desc='I', leave=False):
        train_set   = scores[best_train_start:train_end]

        elos_initial = eloRatings(train_set,best_K,I,best_H)[0]
        elo_loss     = elo_log_loss(elos_initial,validation_set,best_K,I,best_H)

        if elo_loss < best_log_loss:
            best_log_loss = elo_loss
            best_I=I
    print('Best I:     {1:.4f} {0:d}'.format(different_type_weights.index(best_I), best_log_loss))
    # print(best_I)

            ERROR  VALUE
Best K:     0.4066 50
Best start: 0.4066 1974-1-1
Best H:     0.4045 100
Best I:     0.4045 0


The error of the forecasting system based on the estimated rating is `0.40`, lower than the error of random guessing of `0.48`. Note that validation shows that accounting for match importance does **not** improve the performance of the system.

In [46]:
full_train_set = scores[best_train_start:validation_end].copy()
elos, elo1, elo2 = eloRatings(full_train_set, K=best_K, H=best_H, I=best_I, return_elo_columns=True)
print('Size of the full training set (train+validation): {0:d} matches'.format(full_train_set.shape[0]))
print('Size of the test set (EURO 2016): {0:d} matches\n'.format(test_set.shape[0]))

full_train_set['elo1'] = elo1
full_train_set['elo2'] = elo2

test_loss = elo_log_loss(elos, test_set, K=best_K, H=best_H, I=best_I)
print('Log loss on the test set:\n   ELO={0:.4f}\nRANDOM={1:.4f}'.format(test_loss, random_guess_loss))

Size of the full training set (train+validation): 30083 matches
Size of the test set (EURO 2016): 51 matches

Log loss on the test set:
   ELO=0.5058
RANDOM=0.4771


In [47]:
for team, elo in top_elo_teams(elos, k=25):
    print('{0:25s} = {1:.1f}'.format(team, elo))

Brazil                    = 2404.7
Argentina                 = 2380.4
Spain                     = 2334.0
Germany                   = 2281.7
Colombia                  = 2278.2
Uruguay                   = 2274.4
Mexico                    = 2261.7
France                    = 2260.7
Chile                     = 2259.7
England                   = 2253.3
Netherlands               = 2223.1
Ecuador                   = 2190.2
Japan                     = 2186.5
United States             = 2170.2
Belgium                   = 2166.6
Portugal                  = 2162.9
Croatia                   = 2149.0
Italy                     = 2147.1
South Korea               = 2144.9
Australia                 = 2126.0
Costa Rica                = 2116.6
Iran                      = 2111.1
Turkey                    = 2109.6
Ukraine                   = 2097.2
Peru                      = 2092.8


### Building the goal prediction model

We use the training data and the Elo ratings to build the goal prediction model. Using Poisson regression, we estimate the effect of the strength of a given team and its opponent on the number of goals scored. We simulate goals scored with the predicted number of goals as $\lambda$, the parameter of the Poisson distribution.

Note that each match generates two examples for Poisson regression.

In [48]:
scores_reversed = full_train_set[['elo2','elo1','goals2']]
scores_reversed.columns = ['elo1', 'elo2', 'goals1']

elo_goals = full_train_set[['elo1','elo2','goals1']].append(scores_reversed)
elo_goals.columns = ['elo_team', 'elo_opponent', 'goals_team']

elo_goals.head()

,elo_team,elo_opponent,goals_team
date,,,
1974-01-17,1500.000000,1500.000000,1
1974-01-18,1487.198700,1500.000000,2
1974-01-20,1503.072973,1516.001625,1
1974-01-25,1500.000000,1500.000000,1
1974-01-02,1500.000000,1500.000000,2


In [49]:
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import Independence
from statsmodels.genmod.families import Poisson

In [50]:
poisson_regression = GEE.from_formula("goals_team ~ elo_team + elo_opponent", data=elo_goals, 
                                      groups=list(range(0, full_train_set.shape[0])) * 2, # Two examples from a match form 
                                                                                          # a group in Poisson regression
                                                                                          # (not used in this project)
                                      cov_struct=Independence(), family=Poisson())
goals_predictor = poisson_regression.fit()
print(goals_predictor.summary())

                               GEE Regression Results                              
Dep. Variable:                  goals_team   No. Observations:                60166
Model:                                 GEE   No. clusters:                    30083
Method:                        Generalized   Min. cluster size:                   2
                      Estimating Equations   Max. cluster size:                   2
Family:                            Poisson   Mean cluster size:                 2.0
Dependence structure:         Independence   Num. iterations:                     7
Date:                     Mon, 31 Jul 2017   Scale:                           1.000
Covariance type:                    robust   Time:                         11:35:01
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8964      0.043     21.010      0.000       0.813       0.980
e

In [51]:
import numpy as np

In [52]:
def simulate_goals(elo_team, elo_opponent):
    goals_lambda = goals_predictor.predict(pd.DataFrame([{'elo_team': elo_team, 'elo_opponent': elo_opponent}]))
    return np.random.poisson(goals_lambda)

def simulate_match(elo1, elo2):
    return simulate_goals(elo1, elo2), simulate_goals(elo2, elo1)

Even though we do not enforce that win probabilities resulting from Poisson simulations are consistent with the probabilities directly predicted by the Elo ratings, they are remarkably close.

In [53]:
elo1, elo2 = elos['Bulgaria'], elos['Greece']
elo_prediction = probabilities(expected_outcome(elo1, elo2))

def outcome_tuple(pair_of_goals):
    return outcome(pair_of_goals[0], pair_of_goals[1])

N=1000
poisson_outcomes, poisson_counts = np.unique(np.array([outcome_tuple(simulate_match(elo1, elo2)) for i in range(0, N)]), 
                                             return_counts=True)
poisson_frequencies = poisson_counts / N
poisson_prediction = {}
for o in range(0, np.size(poisson_outcomes)):
    poisson_prediction[poisson_outcomes[o]] = poisson_frequencies[o]

for o,p in elo_prediction.items():
    print('{0:.1f} ELO={1:.2f} POISSON={2:.2f}'.format(o, p, poisson_prediction[o]))

1.0 ELO=0.29 POISSON=0.33
0.5 ELO=0.33 POISSON=0.28
0.0 ELO=0.38 POISSON=0.39


## Building the simulator

The simulator comprises two major components. The first component simulates the group stage, including the tiebreaking rules. The second component simulates the knockout stage

### Simulating the group stage
See Wikipedia ([1](https://en.wikipedia.org/wiki/UEFA_Euro_2016#Tiebreakers), 
[2](https://en.wikipedia.org/wiki/UEFA_Euro_2016#Ranking_of_third-placed_teams),
[3](https://en.wikipedia.org/wiki/UEFA_Euro_2016_knockout_phase#Format)) 
for the tournament scheme and the description of tie-breaking rules.

In [54]:
group_country = [
    ['France','Romania','Albania','Switzerland'],
    ['England','Russia','Wales','Slovakia'],
    ['Germany','Ukraine','Poland','Northern Ireland'],
    ['Spain','Czechia','Turkey','Croatia'],
    ['Belgium','Italy','Ireland','Sweden'],
    ['Portugal','Iceland','Austria','Hungary']
]

In [55]:
from functools import cmp_to_key

In [24]:
# Basic group stage functions
def team_pairs(teams):
    pairs = []
    for i in range(0, len(teams)):
        for j in range(i+1, len(teams)):
            pairs.append([teams[i], teams[j]])
    return pairs

def simulate_match_result(team1, team2, elos):
    goals1, goals2 = simulate_match(elos[team1], elos[team2])
    return [team1, team2, goals1, goals2]

In [25]:
# Processing group results
def points_team(goals_team, goals_opp):
    if (goals_team > goals_opp):
        return 3
    elif goals_team == goals_opp:
        return 1
    else:
        return 0

def calculate_points_difference_goals(teams, match_results):
    points = {}
    difference = {}
    goals = {}
    
    for t in teams:
        points[t] = 0
        difference[t] = 0
        goals[t] = 0
        
    for r in match_results:
        t1,t2,g1,g2 = r
        points[t1] = points[t1] + points_team(g1, g2)
        points[t2] = points[t2] + points_team(g2, g1)
        difference[t1] = difference[t1] + (g1 - g2)
        difference[t2] = difference[t2] + (g2 - g1)
        goals[t1] = goals[t1] + g1
        goals[t2] = goals[t2] + g2

    return points, difference, goals

def tied_teams(points):
    ties = {}
    for t,ps in points.items():
        if ps in ties:
            ties[ps].append(t)
        else:
            ties[ps] = [t]
            
    return {ps:teams for ps,teams in ties.items() if len(teams) > 1}

def process_ties(ties, match_results):
    points_within_ties = {}
    difference_within_ties = {}
    goals_within_ties = {}
    
    for _, tie_teams in ties.items():
        results_within_tie = [r for r in match_results if r[0] in tie_teams and r[1] in tie_teams]
        points_within_tie, difference_within_tie, goals_within_tie = calculate_points_difference_goals(tie_teams, results_within_tie)
        
        for t in tie_teams:
            points_within_ties[t] = points_within_tie[t]
            difference_within_ties[t] = difference_within_tie[t]
            goals_within_ties[t] = goals_within_tie[t]
        
    return points_within_ties, difference_within_ties, goals_within_ties

In [26]:
# Simulation

## To simplify reading the code
TEAM_NAME=0
POINTS=1
DIFF=2
GOALS=3
GROUP=4

def simulate_group(teams, elos):
    results = [simulate_match_result(t1, t2, elos) for t1,t2 in team_pairs(teams)]
    
    points, diff, goals = calculate_points_difference_goals(teams,results)
    ties = tied_teams(points)
    
    if ties: # If `ties` isn't empty, calculate advanced tiebreakers
        points_within_ties, difference_within_ties, goals_within_ties = process_ties(ties, results)    
        def compare_teams(t1, t2):
            if points[t1] != points[t2]:
                return points[t1] - points[t2]
            elif points_within_ties[t1] - points_within_ties[t2]:
                return points_within_ties[t1] - points_within_ties[t2]
            elif difference_within_ties[t1] != difference_within_ties[t2]:
                return difference_within_ties[t1] - difference_within_ties[t2]
            elif goals_within_ties[t1] - goals_within_ties[t2]:
                return goals_within_ties[t1] - goals_within_ties[t2]
            elif diff[t1] != diff[t2]:
                return diff[t1] - diff[t2]
            elif goals[t1] != goals[t2]:
                return goals[t1] - goals[t2]
            else:
                return 0 # Sometimes no tiebreakers apply
            
        sorted_teams=sorted(teams,key=cmp_to_key(compare_teams), reverse=True)
    else:
        sorted_teams=sorted(teams, key=lambda t: points[t], reverse=True)
    
    return [[t, points[t], diff[t], goals[t]] for t in sorted_teams]

def simulate_group_stage(groups, elos):
    group_rankings = [simulate_group(g, elos) for g in groups]
    
    group_stage_results = {}
    for i in range(0, len(groups)):
        group_stage_results[str(i) + '0'] = group_rankings[i][0][TEAM_NAME]
        group_stage_results[str(i) + '1'] = group_rankings[i][1][TEAM_NAME]
        
    third_placed_teams = [group_rankings[i][2] + [i] for i in range(0, len(groups))]
    sorted_3rd_placed_teams = sorted(third_placed_teams, reverse=True,
                                     key=lambda team_stats_list: (team_stats_list[POINTS], team_stats_list[DIFF], team_stats_list[GOALS]))
    for t in sorted_3rd_placed_teams[0:4]:
        group_stage_results[str(t[GROUP]) + '2'] = t[TEAM_NAME]
    
    return group_stage_results

Simulate the group stage once:

In [57]:
group_letters = ['A', 'B', 'C', 'D', 'E', 'F']
qualified = simulate_group_stage(group_country, elos)

for code in sorted(qualified.keys()):
    group = group_letters[int(code[0])]
    rank  = int(code[1]) + 1
    
    print('{0:1s}{1:1d} {2:s}'.format(group, rank, qualified[code]))

A1 Switzerland
A2 Albania
A3 Romania
B1 England
B2 Slovakia
B3 Russia
C1 Germany
C2 Ukraine
C3 Poland
D1 Spain
D2 Turkey
D3 Croatia
E1 Sweden
E2 Ireland
F1 Portugal
F2 Iceland


In [58]:
# If necessary for testing
actual_group_results = {
    '00': 'France',
    '01': 'Switzerland',
    '10': 'Wales',
    '11': 'England',
    '12': 'Slovakia',
    '20': 'Germany',
    '21': 'Poland',
    '22': 'Northern Ireland',
    '30': 'Croatia',
    '31': 'Spain',
    '40': 'Italy',
    '41': 'Belgium',
    '42': 'Republic of Ireland',
    '50': 'Hungary',
    '51': 'Iceland',
    '52': 'Portugal',
}

### Simulating the knockout stage

In [59]:
from random import random

In [60]:
A,B,C,D,E,F = 0,1,2,3,4,5

def third_placed(group_results):
    '''Return the comma-separated indices of the group, from which the 3rd-placed teams did qualify'''
    return ','.join(sorted([ k[0] for k, _ in group_results.items() if k[1] == '2' ]))

group_combinations = {
'0,1,2,3': [C,D,A,B],
'0,1,2,4': [C,A,B,E],
'0,1,2,5': [C,A,B,F],
'0,1,3,4': [D,A,B,E],
'0,1,3,5': [D,A,B,F],
'0,1,4,5': [E,A,B,F],
'0,2,3,4': [C,D,A,E],
'0,2,3,5': [C,D,A,F],
'0,2,4,5': [C,A,F,E],
'0,3,4,5': [D,A,F,E],
'1,2,3,4': [C,D,B,E],
'1,2,3,5': [C,D,B,F],
'1,2,4,5': [E,C,B,F],
'1,3,4,5': [E,D,B,F],
'2,3,4,5': [C,D,F,E]
}

def choose_3rd(opponent_group, all_third_placed):
    '''See the 3rd link to the Wikipedia above'''
    return str(group_combinations[all_third_placed][opponent_group]) + '2'
    
def create_bracket(group_results):
    gr = group_results
    tp = third_placed(group_results)
    
    return [(gr['01'], gr['21']),
            (gr['30'], gr[choose_3rd(opponent_group=3, all_third_placed=tp)]),
            (gr['10'], gr[choose_3rd(opponent_group=1, all_third_placed=tp)]),
            (gr['50'], gr['41']),
            (gr['20'], gr[choose_3rd(opponent_group=2, all_third_placed=tp)]),
            (gr['40'], gr['31']),
            (gr['00'], gr[choose_3rd(opponent_group=0, all_third_placed=tp)]),
            (gr['11'], gr['51'])]

def elo_probability(elo1, elo2):
    return expected_outcome(elo1, elo2)

def simulate_winner(teams, elo):
    team1 = teams[0]
    team2 = teams[1]
    elo1 = elo.get(team1,0)
    elo2 = elo.get(team2,0)
    
    probability_1_wins = elo_probability(elo1, elo2)
    winner = team1 if random() < probability_1_wins else team2
    return winner

def simulate_bracket(bracket, elo):
    new_bracket = bracket
    while True:
        winners = [simulate_winner(match, elo) for match in new_bracket]
        if len(winners) == 1:
            return winners[0]
        else:
            new_bracket = [(winners[2*i], winners[2*i+1]) for i in range(0, len(winners) // 2)]

def simulate_knockout(group_results, elo):
    '''Simulates the knockout phase and returns the winner of the tournament'''
    bracket = create_bracket(group_results)    
    winner = simulate_bracket(bracket, elo)
    return winner

### Simulator

In [61]:
def simulate_tournament():
    knockout_bracket = simulate_group_stage(group_country, elos)
    winner = simulate_knockout(knockout_bracket, elos)
    return winner

## Computing the title probabilities

In [63]:
winners = []
max_iterations=1000

N = 0
while N < max_iterations:
    winners.append(simulate_tournament())
    N = N+1
    
    if (N<100 and N%10==0) or (N<1000 and N%100==0) or (N>=1000 and N%1000==0):
        teams, counts = np.unique(winners, return_counts=True)
        frequencies = counts / N
    
        team_ranking=sorted(range(0, np.size(teams)),key=lambda t:frequencies[t],reverse=True)
        for t in team_ranking:
            print('{2:4d};{0:17s};{1:.4f}'.format(teams[t], frequencies[t], N), flush=True)

  10;Germany          ;0.4000
  10;Austria          ;0.1000
  10;Croatia          ;0.1000
  10;France           ;0.1000
  10;Poland           ;0.1000
  10;Romania          ;0.1000
  10;Russia           ;0.1000
  20;Germany          ;0.3000
  20;England          ;0.1500
  20;Russia           ;0.1000
  20;Spain            ;0.1000
  20;Austria          ;0.0500
  20;Croatia          ;0.0500
  20;France           ;0.0500
  20;Poland           ;0.0500
  20;Portugal         ;0.0500
  20;Romania          ;0.0500
  20;Ukraine          ;0.0500
  30;Spain            ;0.3000
  30;Germany          ;0.2667
  30;England          ;0.1333
  30;Russia           ;0.0667
  30;Austria          ;0.0333
  30;Croatia          ;0.0333
  30;France           ;0.0333
  30;Poland           ;0.0333
  30;Portugal         ;0.0333
  30;Romania          ;0.0333
  30;Ukraine          ;0.0333
  40;Spain            ;0.2750
  40;England          ;0.2000
  40;Germany          ;0.2000
  40;France           ;0.0500
  40;Russi